In [11]:
import pandas as pd
import numpy as np
import matplotlib as mp
import seaborn as sns

In [38]:
path_focii = "C:/Users/Modern/Desktop/Kirill/B8/B8/Images/Counted/MyExpt_H2AX_in_nucleus.csv"
path_nuclei = "C:/Users/Modern/Desktop/Kirill/B8/B8/Images/Counted/MyExpt_Nucleus_filtered.csv"
focii_pd=pd.read_csv(path_focii)
nuclei_pd=pd.read_csv(path_nuclei)
labels_nuclei = nuclei_pd.head()
labels_focii = focii_pd.head()
size = focii_pd.shape[0]
print(labels_nuclei)

   ImageNumber  ObjectNumber
0            1             1
1            1             2
2            1             3
3            1             4
4            1             5


In [39]:
print(nuclei_pd['Children_Masked_H2AX_Count'])

KeyError: 'Children_Masked_H2AX_Count'

In [37]:
# sns.scatterplot(x = data_pd['Intensity_IntegratedIntensity_Red'], y=data_pd['AreaShape_EquivalentDiameter'])
# sns.scatterplot(x = nuclei_pd['Children_Masked_H2AX_Count'],y = nuclei_pd['AreaShape_EquivalentDiameter'])
sns.histplot(data=nuclei_pd, x =nuclei_pd['Children_Masked_H2AX_Count'])
# sns.histplot(data=nuclei_pd, x =focii_pd['Intensity_IntegratedIntensity_Red'])


KeyError: 'Children_Masked_H2AX_Count'